# Unofficial ACI Guide

## Python 3 Get Global Endpoints Example

This is a simple Python example demonstrating how to obtain Global Endpoints list.
In the GUI, this is done through Fabric -> Inventory -> Topology -> Global End-Points.

In [1]:
import requests
import json
import pandas as pd

We'll read the auth info from config.py. 
Just edit that file to include your controller IP/hostname, username, and password.

In [2]:
from config import controller, username, password

In [17]:
# Set some variables to construct our login URLs

base_url = "https://" + str(controller) + "/api/"
auth_bit = "aaaLogin.json"

auth_url = base_url + auth_bit
auth_url

In [4]:
# JSON auth data 
auth_data = {
  "aaaUser":{
    "attributes":{
      "name":username,
      "pwd":password
    }
  }
}

## Construct the Request

In [5]:
# Uncomment this to suppress certificate warnings.
# requests.packages.urllib3.disable_warnings() 

In [16]:
# We use verify=False to allow self-signed certificates. 
# Change to "verify=True" or just remove verify=False if you use an internal CA or a valid chain cert.
s = requests.session()
s.post(auth_url, json=auth_data, verify=False)

/anaconda2/envs/aci/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


<Response [200]>

In [7]:
# https://apic-ip-address/api/node/class/fvCEp.xml
# What's the endpoints URL?
epo="node/class/fvCEp.json"
ep_url = base_url + epo
ep_url

'https://10.18.188.101/api/node/class/fvCEp.json'

In [8]:
s_response = s.get(ep_url, verify=False)
s_ep = s_response.json()

/anaconda2/envs/aci/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [18]:
# Let's print out formatted JSON response for easy viewing
print(json.dumps(s_ep, indent=4, sort_keys=True))

{
    "imdata": [
        {
            "fvCEp": {
                "attributes": {
                    "annotation": "",
                    "childAction": "",
                    "contName": "",
                    "dn": "uni/tn-COAST/ctx-COAST_vrf/cep-00:DE:FB:79:8D:43",
                    "encap": "vlan-222",
                    "extMngdBy": "",
                    "id": "0",
                    "idepdn": "",
                    "ip": "192.168.50.12",
                    "lcC": "learned",
                    "lcOwn": "local",
                    "mac": "00:DE:FB:79:8D:43",
                    "mcastAddr": "not-applicable",
                    "modTs": "2019-02-28T15:15:23.544-05:00",
                    "monPolDn": "uni/tn-common/monepg-default",
                    "name": "00:DE:FB:79:8D:43",
                    "nameAlias": "",
                    "status": "",
                    "uid": "0",
                    "uuid": "",
                    "vmmSrc": ""
                }
    

Explanation of the output above:  
 - The imdata object is a dictionary key. Its value is a list of objects (list elements) returned from ACI.   
 - Each object (list element) can be referenced via the list index, starting with element 0.   
 - Each list element itself is a dictionary, with the key being the class name "fvCEp" and the value being a  dictionary with the key called "attributes", with the value being a dictionary of key/value pairs.    
 
Confused yet? 

## Extract DN from each object (example)

In [10]:
# Using list element zero [0] for imdata, we'll extract the distinguished name.

# How to split out the DN in each object.
# s_ep has a dictionary key of 'imdata', which value is a list and we want element 0 in that list. 
# In element 0, we specify the dictionary key (fvCEp) to get the value, which is a dictionary names attributes, 
# so we specify that key to get the values, which are dictionaries. The dictionary key we want ther is "dn".
# Phew!

dn = s_ep['imdata'][0]['fvCEp']['attributes']['dn']

# We'll split() the string using the forward slash as a separator; this creates a list from the string value, 
# and we'll assign it to split_dn. Now we can access each element in the DN as a separate list element.
split_dn = dn.split("/")
universe = split_dn[0]
tenant = split_dn[1]
epg = split_dn[2]

# The endpoint is always the last item in the list, which can be accessed as [-1]. 
# The second item could be referenced as -2, and so forth. 
# The reason we do this is because the third element ([2], since we start counting at zero)
# may be an application profile, putting the EPG into element [3] or it may be a VRF which means 
# there is no application profile to contain the EPG, putting it at [2]. Crazy, right?
# We'll ignore app profiles below and just worry about EPGs.

ep = split_dn[-1]
print("DN: " + str(dn))
print("Tenant: " + str(tenant))
print("EPG or AP: " + str(epg))
print("Endpoint: " + str(ep))
print("Note: the 'EPG or AP' field may be an endpoint group or an application profile")

DN: uni/tn-COAST/ctx-COAST_vrf/cep-00:DE:FB:79:8D:43
Tenant: tn-COAST
EPG or AP: ctx-COAST_vrf
Endpoint: cep-00:DE:FB:79:8D:43
Note: the 'EPG or AP' field may be an endpoint group or an application profile


## Extracting desired info from each object

In [11]:
# Set up some empty lists for tenant, endpoint group, endpoint, IP address, MAC address, and encapsulation. 
tenant_list = []
epg_list = []
ep_list = []
ip_list = []
mac_list = []
encap_list = []

# The imdata dictionary key has a single value which is a list of objects
# that we'll extract into "object_list" for easier ereferenece... 
object_list = s_ep['imdata']

# ...and iterate over it to retrieve the dictionary values we want.
# This is a bit simpler than accessing each element directly like we did above. 
# We're printing the object on each iteration for convenience.

for object in object_list:
    print(object)
    #dn = object['fvCEp']['attributes']['dn']
    dn = object['fvCEp']['attributes']['dn']
    split_dn = dn.split("/")
    tenant_list.append(split_dn[1])
    epg_list.append(split_dn[-2])
    ep_list.append(split_dn[-1])
    ip_list.append(object['fvCEp']['attributes']['ip'])
    mac_list.append(object['fvCEp']['attributes']['mac'])
    encap_list.append(object['fvCEp']['attributes']['encap'])


{'fvCEp': {'attributes': {'annotation': '', 'childAction': '', 'contName': '', 'dn': 'uni/tn-COAST/ctx-COAST_vrf/cep-00:DE:FB:79:8D:43', 'encap': 'vlan-222', 'extMngdBy': '', 'id': '0', 'idepdn': '', 'ip': '192.168.50.12', 'lcC': 'learned', 'lcOwn': 'local', 'mac': '00:DE:FB:79:8D:43', 'mcastAddr': 'not-applicable', 'modTs': '2019-02-28T15:15:23.544-05:00', 'monPolDn': 'uni/tn-common/monepg-default', 'name': '00:DE:FB:79:8D:43', 'nameAlias': '', 'status': '', 'uid': '0', 'uuid': '', 'vmmSrc': ''}}}
{'fvCEp': {'attributes': {'annotation': '', 'childAction': '', 'contName': '', 'dn': 'uni/tn-COAST/ctx-COAST_vrf/cep-00:DE:FB:79:8B:C3', 'encap': 'vlan-50', 'extMngdBy': '', 'id': '0', 'idepdn': '', 'ip': '192.168.50.11', 'lcC': 'learned', 'lcOwn': 'local', 'mac': '00:DE:FB:79:8B:C3', 'mcastAddr': 'not-applicable', 'modTs': '2019-02-15T12:34:31.824-05:00', 'monPolDn': 'uni/tn-common/monepg-default', 'name': '00:DE:FB:79:8B:C3', 'nameAlias': '', 'status': '', 'uid': '0', 'uuid': '', 'vmmSrc':

In [12]:
# Let's take our list and zip them up.
list_of_ep = zip(tenant_list, epg_list, ep_list, ip_list, mac_list, encap_list)

In [13]:
# Convert to list of tuples and stuff it into a Pandas dataframe...
df_input = list(list_of_ep)
df = pd.DataFrame(df_input, columns=("Tenant","EPG or VRF/L3out","Endpoint","IP","MAC Addy","Encap"))
# Note, if you change the column order in the dataframe, be sure to change order of 
# lists zipped into list_of_ep above.

In [14]:
# Print the dataframe in Jupyter.
df

# Use this to print in a standalone script.
#print(df)

,Tenant,EPG or VRF/L3out,Endpoint,IP,MAC Addy,Encap
0,tn-COAST,ctx-COAST_vrf,cep-00:DE:FB:79:8D:43,192.168.50.12,00:DE:FB:79:8D:43,vlan-222
1,tn-COAST,ctx-COAST_vrf,cep-00:DE:FB:79:8B:C3,192.168.50.11,00:DE:FB:79:8B:C3,vlan-50
2,tn-COAST,epg-epg1b,cep-00:50:56:93:6C:74,101.1.1.102,00:50:56:93:6C:74,vlan-1102
3,tn-COAST,ctx-COAST_vrf,cep-00:22:BD:F8:19:FF,192.168.50.1,00:22:BD:F8:19:FF,vlan-50
4,tn-COAST,ctx-COAST_vrf,cep-00:00:00:00:00:00,100.64.3.1,00:00:00:00:00:00,vlan-50
5,tn-COAST,G-PBR_DEVctxCOAST_vrf-N-pbr-C-1arm,cep-00:DE:FB:79:8B:C3,100.64.4.1,00:DE:FB:79:8B:C3,vlan-223


In [15]:
# Let's extract just the tenants and endpoints, create a groupby object, 
# then count the number of endpoints per tenant.
ep = df[['Tenant','Endpoint']]
grouped_df = ep.groupby(['Tenant']).count()
grouped_df

# Use this to print in a standalone script.
#print(grouped_df)

,Endpoint
Tenant,
tn-COAST,6
